In [2]:
!pip install imblearn

    100% |████████████████████████████████| 174kB 787kB/s ta 0:00:01


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier 
from sklearn.naive_bayes import GaussianNB 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis 
from sklearn.model_selection import cross_val_score
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE # 过抽样处理库SMOTE
from imblearn.ensemble import EasyEnsemble # 简单集成方法EasyEnsemble
import seaborn as sns
import pandas as pd


rpeak_train = pd.read_csv("./data/rpeakfeature.csv", header =0,index_col=0)
other_train = pd.read_csv("./data/otherfeature_complete.csv", header =0,index_col=0)
hrv_train = pd.read_csv("./data/hrvFeatures.csv",header = None,index_col=0)
bio_train = pd.read_csv("./data/bioFeatures.csv",header=None,index_col=0)

y = pd.read_csv("./data/y_train.csv",header=0,index_col=0)
print("The len of rpeak train: ",len(rpeak_train),
     "\nThe len of other train: ",len(other_train),
     "\nThe len of hrv train: ",len(hrv_train),
     "\nThe len of bio train: ",len(bio_train))
x_train = pd.concat([rpeak_train,other_train,hrv_train,bio_train],axis=1)


The len of rpeak train:  5117 
The len of other train:  5117 
The len of hrv train:  5117 
The len of bio train:  5117


In [4]:
rpeak_test = pd.read_csv("./data/rpeakfeature_test.csv", header =0,index_col=0)
other_test = pd.read_csv("./data/otherfeature_complete_test.csv", header =0,index_col=0)
hrv_test = pd.read_csv("./data/hrvFeatures_test.csv",header = None,index_col=0)
bio_test = pd.read_csv("./data/bioFeatures_test.csv",header=None,index_col=0)

x_test = pd.concat([rpeak_test,other_test,hrv_test,bio_test],axis=1)

In [6]:
model_smote = SMOTE() # 建立SMOTE模型对象
x_over_train, y_over_train = model_smote.fit_sample(x_train,y) # 输入数据并作过抽样处理

# have a view of the size of resampled data
y_over_train =pd.DataFrame(y_over_train,columns=['label'])
y_over_size=y_over_train.groupby('label').size()
print("The over sampling size is ",y_over_size)

/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The over sampling size is  label
0    3030
1    3030
2    3030
3    3030
dtype: int64


In [7]:
print("Spleating the dataset...")
X_train, X_val, Y_train, Y_val = train_test_split(x_train,y,test_size=0.2)
X_over_train, X_over_val, Y_over_train, Y_over_val = train_test_split(x_over_train,y_over_train,test_size=0.2)

Spleating the dataset...


In [8]:
print(len(X_train),"and the len of test: ",len(X_val))
print(len(X_over_train),"and the len of test: ",len(X_over_val))

4093 and the len of test:  1024
9696 and the len of test:  2424


In [19]:
import xgboost as xgb
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
train_X, eval_X, train_Y, eval_Y = train_test_split(x_over_train, y_over_train, test_size=0.33, random_state=42)
xg_train = xgb.DMatrix(train_X, label=train_Y)
xg_eval = xgb.DMatrix(eval_X, label=eval_Y)

# setup parameters for xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eta'] = 0.2
param['gamma'] = 1.0
param['max_depth'] = 6
param['silent'] = 1
param['subsample'] = 0.8
param['colsample_bytree'] = 0.9
param['min_child_weight'] = 20
param['num_class'] = 4


watchlist = [(xg_train, 'train'), (xg_eval, 'eval')]
num_round = 80
bst = xgb.train(param, 
                xg_train, 
                num_round, 
                watchlist, 
                feval=lambda y,t: ("f1", f1_score(y, t.get_label(), average='micro')))

# get prediction
pred = bst.predict(xg_eval)
# error_rate = np.sum(pred != test_Y) / test_Y.shape[0]
F1 = f1_score(eval_Y, pred, average='micro')
print('Test error using softmax = {}'.format(F1))

[0]	train-merror:0.16564	eval-merror:0.1985	train-f1:0.83436	eval-f1:0.8015
[1]	train-merror:0.14803	eval-merror:0.179	train-f1:0.85197	eval-f1:0.821
[2]	train-merror:0.139409	eval-merror:0.1725	train-f1:0.860591	eval-f1:0.8275
[3]	train-merror:0.130788	eval-merror:0.16575	train-f1:0.869212	eval-f1:0.83425
[4]	train-merror:0.125493	eval-merror:0.1595	train-f1:0.874507	eval-f1:0.8405
[5]	train-merror:0.121059	eval-merror:0.15625	train-f1:0.878941	eval-f1:0.84375
[6]	train-merror:0.116626	eval-merror:0.155	train-f1:0.883374	eval-f1:0.845
[7]	train-merror:0.114039	eval-merror:0.1525	train-f1:0.885961	eval-f1:0.8475
[8]	train-merror:0.110714	eval-merror:0.148	train-f1:0.889286	eval-f1:0.852
[9]	train-merror:0.107759	eval-merror:0.145	train-f1:0.892241	eval-f1:0.855
[10]	train-merror:0.103448	eval-merror:0.14225	train-f1:0.896552	eval-f1:0.85775
[11]	train-merror:0.1	eval-merror:0.1415	train-f1:0.9	eval-f1:0.8585
[12]	train-merror:0.095074	eval-merror:0.13625	train-f1:0.904926	eval-f1:0.863

In [23]:
xg_test = xgb.DMatrix(x_test.values)
y_pred = bst.predict(xg_test)
f = open("submission.csv", "w")
f.write("id,y\n")
for i,x in enumerate(y_pred):
    f.write("{},{}\n".format(i,y_pred[i]))
f.close()

In [16]:
def ModelsOnDatasets(datasets, x_train,x_test,y_train,y_test):
    # clfs = [ DecisionTreeClassifier(), RandomForestClassifier(),
    #         GradientBoostingClassifier(), LinearDiscriminantAnalysis()]
    clfs = [GradientBoostingClassifier()]
    print("The Datasets are: ",datasets)
    log_cols=["Classifier", "Accuracy", "Log Loss"]
    log = pd.DataFrame(columns=log_cols) 
    for clf in clfs: 
        clf.fit(x_train, y_train) 
        name = clf.__class__.__name__ 
        print("="*30,'\n', name)
    
        train_predictions = clf.predict(x_test) 
        cross_val_score(clf,x_train,y_train,scoring='f1_micro', cv=5)
        acc = accuracy_score(y_test, train_predictions) 
        
        train_predictions = clf.predict_proba(x_test) 
        ll = log_loss(y_test, train_predictions) 
        
        # write the item in the log
        #log_entry = pd.DataFrame([[name, acc*100, ll, BMAC]], columns=log_cols) 
        log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols) 
        log = log.append(log_entry) 
        
    
        print('****Results****') 
        print("Accuracy: {:.4%}".format(acc)) 
        print("Log Loss: {}".format(ll)) 
        print("="*30)
    return log, clfs
    

def plotLog(log):
    sns.set_color_codes("muted") 
    sns.barplot(x='Accuracy', y='Classifier', data=log, color="b") 

    plt.xlabel('Accuracy %') 
    plt.title('Classifier Accuracy') 
    plt.show() 

    sns.set_color_codes("muted") 
    sns.barplot(x='Log Loss', y='Classifier', data=log, color="g") 

    plt.xlabel('Log Loss') 
    plt.title('Classifier Log Loss') 
    plt.show()


In [10]:

log, clf_original=ModelsOnDatasets("Original Datasets",X_train, X_val, Y_train, Y_val)
plotLog(log)

The Datasets are:  Original Datasets
 DecisionTreeClassifier
****Results****
Accuracy: 74.1211%
Log Loss: 8.938257563136066


/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


 RandomForestClassifier


/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:528: DataConversionWarni

****Results****
Accuracy: 78.1250%
Log Loss: 1.2952648932583524


/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


 GradientBoostingClassifier


/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape

KeyboardInterrupt: 

In [17]:
log_over, clf_over=ModelsOnDatasets("Over Sampling Datasets",X_over_train, X_over_val, Y_over_train, Y_over_val)
# plotLog(log_over)

The Datasets are:  Over Sampling Datasets


/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


 GradientBoostingClassifier


/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


KeyboardInterrupt: 

In [15]:
print("Testing the data...")
y_pred_gb = clf_over[0].predict(x_test)


y_pred_gb = pd.DataFrame(y_pred_gb, columns=['label'])
y_pred_gb.groupby('label').size()

f = open("submission.csv", "w")
f.write("id,y\n")
for i in range(y_pred_gb.shape[0]):
    f.write("{},{}\n".format(i, int(y_pred_gb.iloc[i,0])))
f.close()

Testing the data...
